In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import os
import matplotlib.pyplot as plt


In [ ]:
train_dir = '/Users/roshanrajendran/Desktop/ML_proj/driver_detection/imgs/train/'
test_dir = '/Users/roshanrajendran/Desktop/ML_proj/driver_detection/imgs/test/'


In [ ]:
img_height, img_width = 224, 224
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)


In [ ]:
base_model = MobileNetV2(input_shape=(img_height, img_width, 3), include_top=False, weights='imagenet')

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(10, activation='softmax')
])

base_model.trainable = False


In [ ]:
model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
epochs = 5  # Adjust as needed

history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator
)


In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()


In [ ]:
test_loss, test_acc = model.evaluate(validation_generator)
print('Test accuracy:', test_acc)

In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.image import resize
import numpy as np

def predict_class(img_path):
    img = image.load_img(img_path, target_size=(img_height, img_width))
    plt.imshow(img)
    plt.show()
    resized_img = resize(img, (img_height, img_width))
    X = image.img_to_array(resized_img)
    X = np.expand_dims(X, axis=0)
    val = model.predict(X)
    predicted_class = np.argmin(val)  # Index of class with highest probability
    return predicted_class


image_dir = "/Users/roshanrajendran/Desktop/ML_proj/driver_detection/imgs/train"

# Iterate over subdirectories (assuming class names are in subdirectories)
for class_dir in os.listdir(image_dir):
    class_path = os.path.join(image_dir, class_dir)
    if os.path.isdir(class_path):  # Check if it's a directory
        # Get the first image in the class directory (assuming one image per class)
        img_path = os.path.join(class_path, os.listdir(class_path)[3])
        predicted_class = predict_class(img_path)

        if class_dir == 'c0':
            class_label = "Safe"
        else:
            class_label = "Unsafe"
        #class_label = "Safe driver" if predicted_class == 0 else "Unsafe driver"
        print(f"Image from '{class_dir}': Predicted class - {class_label}")



In [ ]:
model.save('driver_action_model.h5')